In [1]:
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os

project_root_dir = "/data1/1shared/jinjiarui/run/Diffusion-TS"
os.chdir(project_root_dir)

from engine.solver import Trainer
from data.build_dataloader import build_dataloader_cond
from utils.io_utils import load_yaml_config, instantiate_from_config
from models.interpretable_diffusion.model_utils import unnormalize_to_zero_to_one

/home/jinjiarui/anaconda3/envs/diffusionts/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class Args_Example:
    def __init__(self) -> None:
        self.gpu = 0
        self.config_path = "./config/sines.yaml"
        self.save_dir = "./experiments/toy_exp"
        self.mode = "synthesis_channel"
        self.synthesis_channels = list(range(1, 5))
        self.milestone = 10
        os.makedirs(self.save_dir, exist_ok=True)


args = Args_Example()
configs = load_yaml_config(args.config_path)
device = torch.device(f"cuda:{args.gpu}" if torch.cuda.is_available() else "cpu")

print(f"args.mode: {args.mode}")

dl_info = build_dataloader_cond(configs, args)
model = instantiate_from_config(configs["model"]).to(device)
trainer = Trainer(config=configs, args=args, model=model, dataloader=dl_info)

trainer.load(args.milestone)

args.mode: synthesis_channel
[1, 2, 3, 4]


Sampling sine-dataset: 100%|██████████| 1000/1000 [00:00<00:00, 7658.38it/s]


In [3]:
dataloader, dataset = dl_info["dataloader"], dl_info["dataset"]
coef = configs["dataloader"]["test_dataset"]["coefficient"]
stepsize = configs["dataloader"]["test_dataset"]["step_size"]
sampling_steps = configs["dataloader"]["test_dataset"]["sampling_steps"]
seq_length, feature_dim = dataset.window, dataset.var_num

In [4]:
samples, ori_data, masks = trainer.restore(
    dataloader, [seq_length, feature_dim], coef, stepsize, sampling_steps
)

if dataset.auto_norm:
    samples = unnormalize_to_zero_to_one(samples)

conditional sampling loop time step:   0%|          | 0/200 [00:00<?, ?it/s]

conditional sampling loop time step: 100%|██████████| 200/200 [00:06<00:00, 28.69it/s]
